In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from data_process import tokenizers, utils
from nanoGPT.model import  GPT, GPTConfig
from lightning_training import LightningGPT, GamesDataModule, GamesDataset
import pytorch_lightning as pl
import torch
import os
import numpy as np
import seaborn as sns
from data_process.utils import add_elo_token_to_games, join_material_tokens

In [3]:
headers = ["index", "id", "date", "white_elo", "black_elo", "result", "ply", "ply_30s", "piece_uci"]

games_df = pd.read_csv("./data/csv/uniform_elo_distribution/test.csv", delimiter=";", header=None, names=headers)

games_df

,index,id,date,white_elo,black_elo,result,ply,ply_30s,piece_uci
0,74497,?,2021.12.05,2523,2684,0-1,52,-1,Pd2d4 39 39 Pd7d5 39 39 Nb1c3 39 39 Pc7c6 39 3...
1,74505,?,2021.12.05,2623,2581,0-1,96,-1,Pd2d4 39 39 Ng8f6 39 39 Pc2c4 39 39 Pe7e6 39 3...
2,74508,?,2021.12.05,2757,2670,1/2-1/2,106,-1,Pd2d4 39 39 Pd7d5 39 39 Ke1d2 39 39 Pe7e5 39 3...
3,74510,?,2021.12.05,2627,2666,0-1,96,-1,Pd2d4 39 39 Ng8f6 39 39 Bc1f4 39 39 Pd7d5 39 3...
4,74517,?,2021.12.05,2666,2571,1-0,47,-1,Pe2e4 39 39 Pc7c5 39 39 Ng1f3 39 39 Nb8c6 39 3...
...,...,...,...,...,...,...,...,...,...
99995,14202188,CJayVtXh,2024.12.05,1303,879,1-0,33,-1,Pe2e4 39 39 Pe7e6 39 39 Ng1f3 39 39 Pf7f6 39 3...
99996,14202189,XkroYjvj,2024.12.05,1099,1209,0-1,59,-1,Pe2e4 39 39 Pe7e5 39 39 Ng1f3 39 39 Nb8c6 39 3...
99997,14202191,DWLSh6Wn,2024.12.05,1070,1019,0-1,48,-1,Pe2e4 39 39 Pe7e5 39 39 Ng1f3 39 39 Ng8f6 39 3...
99998,14202194,g7tUv4Rt,2024.12.05,1211,1173,1-0,9,-1,Pe2e4 39 39 Pd7d5 39 39 Bf1c4 39 39 Pe7e5 39 3...


In [4]:
games = join_material_tokens(games_df.piece_uci, replace_bigger_values=True)
games = add_elo_token_to_games(games, games_df.white_elo, games_df.black_elo)

In [5]:
tokenizer = tokenizers.FullMoveEloMaterialPairTokenizer()

model_config = GPTConfig(
    block_size=604,
    vocab_size=tokenizer.vocab_size,
    n_layer=8,
    n_head=8,
    n_embd=512,
    bias=False,
)

test_start_token = 1


material_checkpoint = "./models/elo_with_material_pair/epoch=4-step=78125.ckpt"

material_model = LightningGPT.load_from_checkpoint(material_checkpoint, config=model_config, test_start_token=test_start_token)

number of parameters: 28.15M


In [6]:
trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=10,
    callbacks=[pl.callbacks.RichProgressBar()],
    precision="bf16-mixed",
)
torch.set_float32_matmul_precision('high')

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [7]:
data_module = GamesDataModule(test_games=list(games), tokenizer=tokenizer, num_workers=8)

In [8]:
trainer.test(material_model, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6979631185531616     │
│         test_loss         │    1.2020225524902344     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.2020225524902344, 'test_acc': 0.6979631185531616}]